In [30]:
import itertools
import random


class Minesweeper():
    """
    Minesweeper game representation
    """

    def __init__(self, height=8, width=8, mines=8):

        # Set initial width, height, and number of mines
        self.height = height
        self.width = width
        self.mines = set()

        # Initialize an empty field with no mines
        self.board = []
        for i in range(self.height):
            row = []
            for j in range(self.width):
                row.append(False)
            self.board.append(row)

        # Add mines randomly
        while len(self.mines) != mines:
            i = random.randrange(height)
            j = random.randrange(width)
            if not self.board[i][j]:
                self.mines.add((i, j))
                self.board[i][j] = True

        # At first, player has found no mines
        self.mines_found = set()

    def print(self):
        """
        Prints a text-based representation
        of where mines are located.
        """
        for i in range(self.height):
            print("--" * self.width + "-")
            for j in range(self.width):
                if self.board[i][j]:
                    print("|X", end="")
                else:
                    print("| ", end="")
            print("|")
        print("--" * self.width + "-")

    def is_mine(self, cell):
        i, j = cell
        return self.board[i][j]

    def nearby_mines(self, cell):
        """
        Returns the number of mines that are
        within one row and column of a given cell,
        not including the cell itself.
        """

        # Keep count of nearby mines
        count = 0

        # Loop over all cells within one row and column
        for i in range(cell[0] - 1, cell[0] + 2):
            for j in range(cell[1] - 1, cell[1] + 2):

                # Ignore the cell itself
                if (i, j) == cell:
                    continue

                # Update count if cell in bounds and is mine
                if 0 <= i < self.height and 0 <= j < self.width:
                    if self.board[i][j]:
                        count += 1

        return count

    def won(self):
        """
        Checks if all mines have been flagged.
        """
        return self.mines_found == self.mines

In [23]:

class Sentence():
    """
    Logical statement about a Minesweeper game
    A sentence consists of a set of board cells,
    and a count of the number of those cells which are mines.
    """

    def __init__(self, cells, count):
        self.cells = set(cells)
        self.count = count

    def __eq__(self, other):
        return self.cells == other.cells and self.count == other.count

    def __str__(self):
        return f"{self.cells} = {self.count}"

    def known_mines(self):
        """
        Returns the set of all cells in self.cells known to be mines.
        """
        
        if self.count == len(self.cells):
            return self.cells
        return None
        raise NotImplementedError

    def known_safes(self):
        """
        Returns the set of all cells in self.cells known to be safe.
        """
        if self.count == 0:
            return self.cells
        return None
        raise NotImplementedError

    def mark_mine(self, cell):
        """
        Updates internal knowledge representation given the fact that
        a cell is known to be a mine.
        """
        #If cell is in the sentence, the function should update the sentence so that cell is no longer in the sentence, but still             #represents a logically correct sentence given that cell is known to be a mine.
        if cell in self.cells:
            self.count -= 1
            self.cells.remove(cell)
            return self
        
        #If cell is not in the sentence, then no action is necessary.
        return self
        
        raise NotImplementedError

    def mark_safe(self, cell):
        """
        Updates internal knowledge representation given the fact that
        a cell is known to be safe.
        """
        if cell in self.cells:
            #self.count -= 1
            self.cells.remove(cell)
            return self
        
        #If cell is not in the sentence, then no action is necessary.
        return self
        raise NotImplementedError


In [29]:
abc = Sentence([(1,2),(3,4)],1)
print(abc.known_mines())
print(abc.known_safes())
abc.mark_safe((1,2))

print(abc.known_safes())
print(abc.known_mines())

None
None
None
{(3, 4)}


In [31]:

class MinesweeperAI():
    """
    Minesweeper game player
    """

    def __init__(self, height=8, width=8):

        # Set initial height and width
        self.height = height
        self.width = width

        # Keep track of which cells have been clicked on
        self.moves_made = set()

        # Keep track of cells known to be safe or mines
        self.mines = set()
        self.safes = set()

        # List of sentences about the game known to be true
        self.knowledge = []

    def mark_mine(self, cell):
        """
        Marks a cell as a mine, and updates all knowledge
        to mark that cell as a mine as well.
        """
        self.mines.add(cell)
        for sentence in self.knowledge:
            sentence.mark_mine(cell)

    def mark_safe(self, cell):
        """
        Marks a cell as safe, and updates all knowledge
        to mark that cell as safe as well.
        """
        self.safes.add(cell)
        for sentence in self.knowledge:
            sentence.mark_safe(cell)

    def add_knowledge(self, cell, count):
        """
        Called when the Minesweeper board tells us, for a given
        safe cell, how many neighboring cells have mines in them.

        This function should:
            1) mark the cell as a move that has been made
            2) mark the cell as safe
            3) add a new sentence to the AI's knowledge base
               based on the value of `cell` and `count`
            4) mark any additional cells as safe or as mines
               if it can be concluded based on the AI's knowledge base
            5) add any new sentences to the AI's knowledge base
               if they can be inferred from existing knowledge
        """
        #1
        self.moves_made.add(cell)
        #2
        self.mark_safe(cell)
        #3
        neighbours = []
        for i in range(-1,2):
            if cell[0]+i>=0 or cell[0]+i<=self.height :
                for j in range(-1,2):
                    if cell[1]+j>=0 or cell[0]+i<=self.width:
                         if (i,j) not in self.moves_made:
                             neighbours.append((i,j))
        self.knowledge.append(Sentence(neighbours,count))

        #4
        nSentences = ["flag"]
        while nSentences:
            nSentences=[]
            for sentence in self.knowledge:
                ksafes = sentence.known_safes() 
                if ksafes:
                    self.mark_safe(ksafes)
                
                kmines=sentence.known_mines()
                if kmines:
                    self.markmines(kmines)

            #5
            
            for i in range(len(self.knowledge)):
                
                for j in range(i,len(self.knowledge)):
                    if j.cells<i.cells:
                        nSentences.append(Sentence(i.cells-j.cells,i.count-j.count))

            self.knowledge = self.knowledge+nSentences


        #raise NotImplementedError

    def make_safe_move(self):
        """
        Returns a safe cell to choose on the Minesweeper board.
        The move must be known to be safe, and not already a move
        that has been made.

        This function may use the knowledge in self.mines, self.safes
        and self.moves_made, but should not modify any of those values.
        """
        for safe_move in self.safe:
            if safe_move not in self.moves_made:
                return safe_move
        return None
        raise NotImplementedError

    def make_random_move(self):
        """
        Returns a move to make on the Minesweeper board.
        Should choose randomly among cells that:
            1) have not already been chosen, and
            2) are not known to be mines
        """
        i=random.randint(a, b)
        j=random.randint(a, b)

        while (i,j) in self.moves_made or (i,j) in self.mines:
            i=random.randint(a, b)
            j=random.randint(a, b)
        return(i,j)
        raise NotImplementedError


In [34]:
a=1
for i in range(-1,2):
    for j in range(-1,2):
        print(a+i,a+j)

0 0
0 1
0 2
1 0
1 1
1 2
2 0
2 1
2 2


In [37]:
abc = [1,2,3]
cd=[5]
f = [6]
cd.append(abc)
f+abc
print(cd,f)

[5, [1, 2, 3]] [6]


In [39]:
n = []
if n:
    print(n)
print(n)

[]


In [44]:
print(0+-1>=0 and 0+8<8 )
print(0+-1>=0)  
print(0+8<8)

False
False
False
